In [1]:
import numpy as np
import sklearn 
import subprocess
import re 
from io import StringIO 
from datetime import date, timedelta, datetime 
import sys
import os
import time
import boto3
import pandas as pd
from multiprocessing import Pool
import multiprocessing
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math
import random
from multiprocessing import Pool
import multiprocessing

#!conda install -c conda-forge pygrib -n python3 -y
import pygrib

#!conda install h5py -n python3 -y
import h5py

#!conda install -c ostrokach gzip -n python3 -y
import gzip

import pandas as pd

In [2]:
#Function which returns the total number of figures in a float, and the total number of decimal figures
def precision_and_scale(x):
    max_digits = 14
    int_part = int(abs(x))
    magnitude = 1 if int_part == 0 else int(math.log10(int_part)) + 1
    if magnitude >= max_digits:
        return (magnitude, 0)
    frac_part = abs(x) - int_part
    multiplier = 10 ** (max_digits - magnitude)
    frac_digits = multiplier + int(multiplier * frac_part + 0.5)
    while frac_digits % 10 == 0:
        frac_digits /= 10
    scale = int(math.log10(frac_digits))
    return (magnitude + scale, scale) 

#function to transfer a file from s3 to efs
def s3_to_efs(d, model, hour, field, forecast):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  grb.values.flatten()
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    end = time.time()
    print(end-start)
    commande_rm = 'rm ' + path_grb2
    os.system(commande_rm)
    path_save = '../efs/' + path_grb2
    np.save(path_save,mat_res)
    return(0)

#function to download a file from s3, transfotm it and putting it back to s3
def s3_csvgz_s3(d, model, hour, field, forecast):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    end = time.time()
    print(end-start)
    start = time.time()
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  grb.values.flatten()
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    end = time.time()
    print(end-start)
    start = time.time()
    df = pd.DataFrame(mat_res)
    name_csv = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.csv'
    print(name_csv) 
    path_csv = '../efs/' + name_csv
    df.to_csv(path_csv, index=False)
    print('csv_fini')
    with open(path_csv, 'rb') as f_in, gzip.open(path_csv +  '.gz', 'wb') as f_out:
        f_out.writelines(f_in) 
    name_csv_gz = name_csv + '.gz'
    end = time.time()
    print(end-start)
    s3 = boto3.client('s3')
    key_data = 'remi/' + name_csv_gz
    s3.upload_file(path_csv + '.gz' , bucket_name, key_data)
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    commande_rm_csv = 'rm ' + path_csv
    os.system(commande_rm_csv)  
    commande_rm_csvgz = 'rm ' + path_csv + '.gz'
    os.system(commande_rm_csvgz) 
    return(0)

#function to download a file from s3, transfotm it and putting it back to s3
def s3_hdf5_s3(d, model, hour, field, forecast):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    end = time.time()
    print(end-start)
    start = time.time()
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  grb.values.flatten()
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    end = time.time()
    print(end-start)
    start = time.time()
    name_hdf5 = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.h5'
    path_hdf5 = '../efs/' + name_hdf5
    h5f = h5py.File(path_hdf5, 'w')
    h5f.create_dataset(name_hdf5, data=mat_res)
    h5f.close() 
    end = time.time()
    print(end-start)
    s3 = boto3.client('s3')
    key_data = 'remi/' + name_hdf5
    data = open(path_hdf5,'rb')
    s3.upload_file(path_hdf5, bucket_name, key_data)
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    commande_rm_hdf5 = 'rm ' + path_hdf5
    os.system(commande_rm_hdf5)  
    return(0)

#function to download a file from s3, transfotm it and putting it back to s3
def s3_hdf5gz_s3(d, model, hour, field, forecast):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    end = time.time()
    print(end-start)
    start = time.time()
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  grb.values.flatten()
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    mat_res[np.where(mat_res==9.969209968386869e+36)] = 0
    end = time.time()
    print(end-start)
    start = time.time()
    name_hdf5gz = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.h5.gz'
    path_hdf5gz = '../efs/' + name_hdf5gz
    h5f = h5py.File(path_hdf5gz, 'w')
    h5f.create_dataset(name_hdf5gz, data=mat_res, compression="gzip")
    h5f.close() 
    print(name_hdf5gz)
    print(path_hdf5gz)
    end = time.time()
    print(end-start)
    s3 = boto3.client('s3')
    key_data = 'remi/' + name_hdf5gz
    data = open(path_hdf5gz,'rb')
    s3.upload_file(path_hdf5gz, bucket_name, key_data)
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    commande_rm_hdf5 = 'rm ' + path_hdf5gz
    os.system(commande_rm_hdf5)  
    return(0)

#function to download a file from s3, transfotm it and putting it back to s3
def s3_npzcompressed_s3(d, model, hour, field, forecast):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    end = time.time()
    print(end-start)
    start = time.time()
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  grb.values.flatten()
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    end = time.time()
    print(end-start)
    start = time.time()
    name_npz = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.npz'
    path_npz = '../efs/' + name_npz
    np.savez_compressed(path_npz, mat_res)
    end = time.time()
    print(end-start)
    s3 = boto3.client('s3')
    key_data = 'remi/' + name_npz
    data = open(path_npz,'rb')
    s3.upload_file(path_npz, bucket_name, key_data)
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    commande_rm_npz = 'rm ' + path_npz
    os.system(commande_rm_npz)  
    return(0)

#function to download a file from s3, transfotm it and putting it back to s3
def s3_npy_s3(d, model, hour, field, forecast):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    end = time.time()
    print(end-start)
    start = time.time()
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  grb.values.flatten()
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    end = time.time()
    print(end-start)
    start = time.time()
    name_npy = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.npy'
    path_npy = '../efs/' + name_npy
    np.save(path_npy, mat_res)
    end = time.time()
    print(end-start)
    s3 = boto3.client('s3')
    key_data = 'remi/' + name_npy
    data = open(path_npy,'rb')
    s3.upload_file(path_npy, bucket_name, key_data)
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    commande_rm_npy = 'rm ' + path_npy
    os.system(commande_rm_npy)  
    return(0)

def s3_hdf5gztrunc_s3(d, model, hour, field, forecast, round_number):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    end = time.time()
    print(end-start)
    start = time.time()
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  np.round(grb.values.flatten(),round_number)
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    mat_res[np.where(mat_res==9.969209968386869e+36)] = 0
    end = time.time()
    print(end-start)
    start = time.time()
    name_hdf5gz = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.h5.gz'
    path_hdf5gz = '../efs/' + name_hdf5gz
    h5f = h5py.File(path_hdf5gz, 'w')
    h5f.create_dataset(name_hdf5gz, data=mat_res, compression="gzip")
    h5f.close() 
    print(name_hdf5gz)
    print(path_hdf5gz)
    end = time.time()
    print(end-start)
    s3 = boto3.client('s3')
    key_data = 'remi/' + name_hdf5gz
    data = open(path_hdf5gz,'rb')
    s3.upload_file(path_hdf5gz, bucket_name, key_data)
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    commande_rm_hdf5 = 'rm ' + path_hdf5gz
    os.system(commande_rm_hdf5)  
    return(0)


In [4]:
#Function to load a grib2 file from s3 and putting it back as a hdf5.gz file
def s3_hdf5gztrunc_s3(d, model, hour, field, forecast, round_number):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  np.round(grb.values.flatten(),round_number)
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    mat_res[np.where( (mat_res==9.969209968386869e+36)  |  (mat_res==float('Inf')) )] = 0
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    end = time.time()
    print(end-start)
    start = time.time()
    name_hdf5gz = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.h5.gz'
    path_hdf5gz = '../efs/' + name_hdf5gz
    h5f = h5py.File(path_hdf5gz, 'w')
    h5f.create_dataset(name_hdf5gz, data=mat_res, compression="gzip")
    h5f.close() 
    end = time.time()
    print(end-start)
    s3 = boto3.client('s3')
    key_data = 'remi/' + name_hdf5gz
    data = open(path_hdf5gz,'rb')
    s3.upload_file(path_hdf5gz, bucket_name, key_data)
    commande_rm_hdf5gz = 'rm ' + path_hdf5gz
    os.system(commande_rm_hdf5gz)  
    return(0)

def s3_hdf5gztruncrand_s3(d, model, hour, field, forecast, round_number):
    start = time.time()
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  np.round(grb.values.flatten(),round_number)
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    mat_res[np.where( (mat_res==9.969209968386869e+36)  |  (mat_res==float('Inf')) )] = 0
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    end = time.time()
    print(end-start)
    start = time.time()
    name_hdf5gz = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.h5.gz'
    path_hdf5gz = '../efs/' + name_hdf5gz
    h5f = h5py.File(path_hdf5gz, 'w')
    h5f.create_dataset(name_hdf5gz, data=mat_res, compression="gzip")
    h5f.close() 
    end = time.time()
    print(end-start)
    s3 = boto3.client('s3')
    tirage = random.random()
    if (tirage<=0.8):
        key_data = 'remi/train/' + name_hdf5gz
    else:
        key_data = 'remi/test/' + name_hdf5gz
    data = open(path_hdf5gz,'rb')
    s3.upload_file(path_hdf5gz, bucket_name, key_data)
    commande_rm_hdf5gz = 'rm ' + path_hdf5gz
    os.system(commande_rm_hdf5gz)  
    return(0)

In [ ]:
#Test run time for the function s3_hdf5gztruncrand_s3
hours = np.arange(24)
fxx = [0]
field = 'prs'
model = 'hrrr'
sDATE = date(2017, 1, 1)          # Start date
eDATE = date(2017, 12, 31)          # End date (exclusive)
days = (eDATE-sDATE).days
DATES = [sDATE + timedelta(days=d) for d in range(days)]


start = time.time()
s3_hdf5gztruncrand_s3(DATES[0], model, hours[0], field, fxx[0], 4)
end = time.time()
print(end-start)

In [5]:
#Multi-processing
# hours = np.arange(24)
# fxx = [0]
# field = 'prs'
# model = 'hrrr'
# sDATE = date(2017, 1, 1)          # Start date
# eDATE = date(2017, 12, 31)          # End date (exclusive)
# days = (eDATE-sDATE).days
# DATES = [sDATE + timedelta(days=d) for d in range(days)]

# DATES_try = DATES[0]
# hours_try = hours[0:12]


# n_proc = multiprocessing.cpu_count()

# start = time.time()

# pool = Pool(processes=n_proc-1)
# result = [ pool.apply_async(s3_hdf5gztrunc_s3, args=(DATES_try,model,h,field,fxx[0],4) ) for h in hours_try]

# for i in range(len(result)):
#     print(result[i].get())
    
# end = time.time()
# print(end-start)

In [18]:
#Final execution chunk
def s3_hdf5gztruncrand_s3(d, model, hour, field, forecast, round_number):
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    bucket_name = 'jedi-pub'    
    key_data = 'remi/' + path_grb2
    s3 = boto3.resource('s3')
    s3.Bucket(bucket_name).download_file(key_data, path_grb2)
    mat_res = np.zeros((1905141,686))
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        mat_res[:,j] =  np.round(grb.values.flatten(),round_number)
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    mat_res[np.where( (mat_res==9.969209968386869e+36)  |  (mat_res==float('Inf')) )] = 0
    commande_rm_grib2 = 'rm ' + path_grb2
    os.system(commande_rm_grib2)
    name_hdf5gz = "%s_%s.t%02dz.wrf%sf%02d" % (d.strftime('%Y%m%d'), model, hour, field, forecast) + '.h5.gz'
    path_hdf5gz = '../efs/' + name_hdf5gz
    h5f = h5py.File(path_hdf5gz, 'w')
    h5f.create_dataset(name_hdf5gz, data=mat_res, compression="gzip")
    h5f.close() 
    s3 = boto3.client('s3')
    tirage = random.random()
    if (tirage<=0.8):
        key_data = 'remi/train/' + name_hdf5gz
    else:
        key_data = 'remi/test/' + name_hdf5gz
    data = open(path_hdf5gz,'rb')
    s3.upload_file(path_hdf5gz, bucket_name, key_data)
    commande_rm_hdf5gz = 'rm ' + path_hdf5gz
    os.system(commande_rm_hdf5gz)  
    return(0)


#Multi-processing
hours = np.arange(24)
fxx = [0]
field = 'prs'
model = 'hrrr'
sDATE = date(2017, 1, 1)          # Start date
eDATE = date(2017, 12, 31)          # End date (exclusive)
days = (eDATE-sDATE).days
DATES = [sDATE + timedelta(days=d) for d in range(days)]

all_DATES_hours = [(d,h) for d in DATES for h in hours]
for i in range(16,24):
    all_DATES_hours.remove((date(2017, 2, 28),i))
all_DATES_hours.remove((date(2017, 8, 29),1))   


n_proc = multiprocessing.cpu_count()

start = time.time()

pool = Pool(processes=n_proc-1)
result = [ pool.apply_async(s3_hdf5gztrunc_s3, args=(d,model,h,field,fxx[0],4) ) for d,h in all_DATES_hours]

for i in range(len(result)):
    print(result[i].get())

datetime.date(2017, 2, 28)